In [10]:
import os
import sys
import random
from glob import glob
from copy import deepcopy
from pprint import pprint

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm

# enable use of local modules
repo_dir = '/grad/bwedig/mejiro'
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

from mejiro.instruments.roman import Roman

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

# set HWO directory
hwo_dir = os.path.join(repo_dir, 'hwo')

RTI_PATH = '/nfsdata1/bwedig/STScI/roman-technical-information'

import yaml
from astropy.table import Table
from astropy.table import QTable

In [11]:
roman = Roman(RTI_PATH)

print(roman.version)

1.1.0


In [12]:
ZEROPOINT_PATH = 'data/WideFieldInstrument/Imaging/ZeroPoints/Roman_zeropoints_20240301.ecsv'

zeropoints = QTable.read(os.path.join(RTI_PATH, ZEROPOINT_PATH))
pprint(zeropoints.columns)

<TableColumns names=('detector','element','VegaMag','ABMag','STMag','FLAM','FNU','PHOTLAM','PHOTNU')>


In [13]:
roman.get_zeropoint_magnitude('F062', 'SCA01')

26.57551933086481


In [14]:
ZODIACAL_LIGHT_PATH = 'data/WideFieldInstrument/Imaging/ZodiacalLight/zodiacal_light.ecsv'

zodi = QTable.read(os.path.join(RTI_PATH, ZODIACAL_LIGHT_PATH))
zodi

filter,rate
,ct / pix
str4,float64
F062,0.25
F087,0.251
F106,0.277
F129,0.267
F158,0.244
F184,0.141
F213,0.118
F146,0.781


In [15]:
rate_f062 = zodi[zodi['filter'] == 'F062']['rate'][0]
print(rate_f062)

0.25 ct / pix


In [16]:
THERMAL_BACKGROUND_PATH = 'data/WideFieldInstrument/Imaging/Backgrounds/internal_thermal_backgrounds.ecsv'

thermal = QTable.read(os.path.join(RTI_PATH, THERMAL_BACKGROUND_PATH))
thermal

filter,rate
,ct / pix
str4,float64
F062,0.003
F087,0.003
F106,0.003
F129,0.003
F158,0.048
F184,0.155
F213,4.38
F146,1.03


In [17]:
FILTER_PARAMS_PATH = 'data/WideFieldInstrument/Imaging/FiltersSummary/filter_parameters.ecsv'

filters = QTable.read(os.path.join(RTI_PATH, FILTER_PARAMS_PATH))
filters

filter,wavelength_min,wavelength_max,wavelength_center,width,PSF_FWHM,corner_PSF_n_eff_pixel,corner_PSF_peak_flux,center_PSF_n_eff_pixel,center_PSF_peak_flux
,um,um,um,um,arcsec,pix,dimensionless,pix,dimensionless
str4,float64,float64,float64,float64,float64,float64,float64,float64,float64
F062,0.48,0.76,0.62,0.28,0.058,7.35,0.20341,3.8,0.49536
F087,0.76,0.977,0.869,0.217,0.073,9.35,0.16517,4.04,0.4838
F106,0.927,1.192,1.06,0.265,0.087,10.96,0.1506,4.83,0.44004
F129,1.131,1.454,1.293,0.323,0.106,11.79,0.14863,6.63,0.36874
F158,1.38,1.774,1.577,0.394,0.128,12.63,0.14343,9.65,0.29081
F184,1.683,2.0,1.842,0.317,0.146,17.15,0.11953,15.52,0.21361
F213,1.95,2.3,2.125,0.35,0.169,20.38,0.10831,20.14,0.17052
F146,0.927,2.0,1.464,1.03,0.105,12.18,0.14521,7.37,0.34546


In [18]:
print(roman.get_zeropoint_magnitude('F062', 'SCA01'))
print(roman.get_thermal_background('F062'))
print(roman.get_minimum_zodiacal_light('F062'))
print(roman.get_psf_fwhm('F062'))

      ABMag      
-----------------
26.57551933086481
[0.003] ct / pix
[0.25] ct / pix
[0.058] arcsec
